In [1]:
print("Hello, Plagiaator!")

Hello, Plagiaator!


In [ ]:
!pip3 install selenium
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display

def parseArticle(url):
  req=Request(url=url, headers={'User-Agent': 'Mozilla/5.0'})
  op = urlopen(req)
  rd = op.read()
  op.close
  soup = BeautifulSoup(rd, 'lxml')
  title=soup.find("h1").text
  #print(title)
  textClasses = soup.find_all('div', class_="text flex-row")
  keywordsClass = soup.find_all('a', class_="btn small round modest")
  keywords=[]
  for el in textClasses:
    text=el.text
  for el in keywordsClass:
    #print(el)
    keywords.append(el.text)
  return title, text, keywords

#File full of links to articles from ERR.
with open("articles.txt") as f:
  id_list=[]
  title_list=[]
  text_list=[]
  keywords_list=[]
  i=0
  for url in f:
    print(i)
    title, text, keywords= parseArticle(url)
    id_list.append(i)
    title_list.append(title)
    text_list.append(text)
    keywords_list.append(keywords)
    i+=1
  #print(id_list)
  #print(title_list)
  #print(text_list)
  #print(keywords_list)

  #Convert to pandas dataframe.
  table=pd.DataFrame({'ID':id_list,
                      'Title':title_list,
                      'Text':text_list,
                      'Keywords':keywords_list})
  
  display(table)
